**[SQL Course Home Page](https://www.kaggle.com/learn/SQL)**

---


# Intro

Stack Overflow (stackoverflow.com) is a widely beloved Question and Answer site for technical questions. You'll probably use it yourself as you keep using SQL (or any programming language). 

Their data is publicly available. What cool things do you think it would be useful for?

Here's one idea:
You could set up a service that identifies the Stack Overflow users who have demonstrated expertise with a specific technology by answering related questions about it, so someone could hire those experts for in-depth help.

In this exercise, you'll write the SQL queries that might serve as the foundation for this type of service.

As usual, run the following cell to set up our feedback system before moving on.

In [1]:
# Set up feedack system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex6 import *

# import package with helper functions 
import bq_helper

# create a helper object for this dataset
stack_overflow = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                              dataset_name="stackoverflow")

Using Kaggle's public dataset BigQuery integration.
Using Kaggle's public dataset BigQuery integration.


# Questions

# 1) Explore the Data

Before writing queries or **JOIN** clauses, you'll want to see what tables are available. 

This may be a good time to practice **tab completion** for when you don't remember command names. If you type `stack_overflow.` and then hit tab, you will see a list of methods for the `stack_overflow` object (don't forget the dot before hitting tab.)

In [2]:
# Your code here
list_of_tables = stack_overflow.list_tables()    # get a list of available tables

print(list_of_tables)
q_1.check()

['badges', 'comments', 'post_history', 'post_links', 'posts_answers', 'posts_moderator_nomination', 'posts_orphaned_tag_wiki', 'posts_privilege_wiki', 'posts_questions', 'posts_tag_wiki', 'posts_tag_wiki_excerpt', 'posts_wiki_placeholder', 'stackoverflow_posts', 'tags', 'users', 'votes']


<IPython.core.display.Javascript object>

None

In [3]:
# q_1.solution()

# 2) Review Relevant Tables

If you are interested in people who answer questions on a given topic, the `posts_answers` table is a natural place to look. Run the following cell and look at the output

In [4]:
stack_overflow.head('posts_answers')

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,55832858,None,<p>There may be privacy description <em>(for S...,None,None,0,None,2019-04-24 14:45:59.090000+00:00,None,2019-04-24 14:45:59.090000+00:00,None,None,None,None,7266317,6004032,2,0,None,None
1,55832872,None,<p>How do I set the name of the button to the ...,None,None,3,None,2019-04-24 14:46:52.690000+00:00,None,2019-04-24 14:46:52.690000+00:00,None,None,None,None,5873109,55832746,2,0,None,None
2,55832876,None,<pre><code>go test -v -timeout 30s &lt;path_to...,None,None,0,None,2019-04-24 14:47:09.227000+00:00,None,2019-04-24 14:47:09.227000+00:00,None,None,None,None,5713047,16935965,2,0,None,None
3,55832878,None,<p>I suggest you create one custom pin represe...,None,None,0,None,2019-04-24 14:47:10.297000+00:00,None,2019-04-24 14:47:10.297000+00:00,None,None,None,None,6001090,33622927,2,0,None,None
4,55832879,None,"<p>You have to use <a href=""https://en.cpprefe...",None,None,2,None,2019-04-24 14:47:16.853000+00:00,None,2019-04-24 14:47:16.853000+00:00,None,None,None,None,10765031,55831782,2,0,None,None


It isn't clear yet how to the find users who answered questions on any given topic. But `posts_answers` has a `parent_id` column. If you are familiar with the Stack Overflow site, you might figure out that the `parent_id` is the question each post is answering.

Look at `posts_questions` using the line below.

In [5]:
stack_overflow.head('posts_questions')

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,56412356,S3 Multipart upload with pause and resume func...,<p>I am trying to acheive s3 multipart upload ...,None,0,0,None,2019-06-02 05:09:24.723000+00:00,None,2019-06-02 05:09:24.723000+00:00,None,None,None,None,7225816,None,1,0,<javascript><amazon-s3><stream>,1
1,56318079,Angular Nginx Docker 404,<p>Been driving myself nuts trying to figure t...,None,1,0,None,2019-05-26 23:14:43.403000+00:00,None,2019-05-26 23:21:11.503000+00:00,None,None,None,None,1058951,None,1,0,<angular><docker><nginx><http-status-code-404>,257
2,56136152,Switch structure control for OSC msg,<p>I’m new to SC and the whole music programmi...,None,0,0,None,2019-05-14 18:05:32.077000+00:00,None,2019-05-14 18:05:32.077000+00:00,None,None,None,None,8880735,None,1,0,<supercollider>,2
3,56207713,AWS QuickSight - Smooth line graph/plot,<p>do you know if ''''AWS QuickSight'''' has a...,None,0,0,None,2019-05-19 12:15:57.697000+00:00,None,2019-05-19 12:15:57.697000+00:00,None,None,None,None,11036699,None,1,0,<amazon>,2
4,56250259,I am not able to access kubernetes dash-board ...,<p>I have installed docker and kubernetes in m...,None,0,0,None,2019-05-22 06:03:36.240000+00:00,None,2019-05-22 06:03:36.240000+00:00,None,None,None,None,9493974,None,1,0,<kubernetes-dashboard>,2


Are there any fields that identify what topic or technology each question is about?

If so, how could you find the user ID\'s of users who answered questions about a specific topic?

Think about it, then check the solution by running the code in the next cell.

In [6]:
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
`posts_questions` has a column called `tags` which lists the topics/technologies each question is about.

`posts_answers` has a column called `parent_id` which identifies the ID of the question each answer is responding to.
`posts_answers` also has an `owner_user_id` column which specifies the ID of the user who answered the question.

You can join these two tables to:
- determine the `tags` for each answer, and then
- select the `owner_user_id` of the answers on the desired tag.

This is exactly what you will do over the next few questions.


# 3 Selecting The Right Questions

A lot of this data is text. 

Here is one last technique you'll learn in this course which you can apply to this text:

A **WHERE** clause can limit your results to rows with certain text using the **LIKE** feature. For example, to select just the third row of the `pets` table, we would write

`SELECT * FROM PETS WHERE NAME LIKE 'Ripley'`

![](https://i.imgur.com/Ef4Puo3.png)

You can also use `%` as a "wildcard" for any number of characters. So you can get the third row with 

`SELECT * FROM PETS WHERE NAME LIKE '%ipl%'`

Try this yourself.
Before finding users who have answered questions, write a query that selects the `id`, `title` and `owner_user_id` from the `posts_questions` table. Restrict the results to rows that contain the word **bigquery** in the `tags` column. Include rows where there is other text in addition to the word `bigquery` (e.g. if a row has a tag `bigquery-sql`, your results should include that too).

In [7]:
# Your code here
questions_query = \
"""
SELECT id, title, owner_user_id
FROM `bigquery-public-data.stackoverflow.posts_questions`
WHERE tags LIKE '%bigquery%'
"""

questions_results = stack_overflow.query_to_pandas_safe(questions_query, max_gb_scanned=25) # this query reads a lot of data
print(questions_results.head())
q_3.check()


         id                                              title  owner_user_id
0  50514789  How do I create partitioned table from result ...      9798355.0
1  50629098               Google Cloud Dataprep Import Recipes      7714325.0
2  50427169                    Pcollection for multiple Tables      1115163.0
3  50591099  Big Query SQL - group over all possible levels...      2054629.0
4  50433595  Dataflow BigQuery read does not return correct...       527143.0


<IPython.core.display.Javascript object>

None

In [8]:
# q_3.hint()
# q_3.solution()

# 4 Your First Join
Now that you have a query to select questions on any given topic (in this case, you chose `bigquery`), you can find the answers to those questions with a **JOIN**.  

Write a SQL query that returns the `id`, `body` and `owner_user_id` from the `posts_answers` table for answers to `bigquery` related questions. That is, you should have one row in your results for each answer to a question that has a `bigquery` in the tag.

Here's a reminder of what a **JOIN** looked like in the tutorial
```
SELECT p.Name AS Pet_Name, o.Name as Owner_Name
FROM `bigquery-public-data.pet_records.pets` as p
INNER JOIN `bigquery-public-data.pet_records.owners` as o ON p.ID = o.Pet_ID
```

It may be useful to scroll up and review the results from when you called **head** on `posts_answers` and `posts_questions`.  

In [9]:
from time import time

answers_query = \
"""
SELECT pa.id, pa.body, pa.owner_user_id
    FROM `bigquery-public-data.stackoverflow.posts_questions` AS pq
        INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS pa
            ON pq.id = pa.parent_id
    WHERE pq.tags LIKE '%bigquery%'
"""

answers_results = stack_overflow.query_to_pandas_safe(answers_query, max_gb_scanned=50) # query scans more than 1GB of data, but less than 2.
print(answers_results.head())
q_4.check()

         id                                               body  owner_user_id
0  42692927  <p>It took me a while to find the answer.\nOnc...      2015196.0
1  42702312  <p>This <a href="https://stackoverflow.com/a/4...        33611.0
2  42727478  <blockquote>\n  <p>Why are the audit logs bein...       395927.0
3  42771004  <p>As of release <a href="https://cloud.google...      6875448.0
4  42825282  <p>If I understand you correctly, you want som...      1993206.0


<IPython.core.display.Javascript object>

None

In [10]:
# q_4.hint()
# q_4.solution()

# 5 Answer The Question
You have the merge you need. But you want a list of users who have answered many questions... which requires more work beyond your previous result.

Write a new query that selects data from the `posts_questions` and `posts_answers` tables. The results should have a single row for each user who answered at least one questions with a tag that includes the string `bigquery`. Each row in your results should have two columns:
- a column called `user_id` that contains the `owner_user_id` from the `posts_answers` table
- a column called `number_of_answers` that contains the number of answers the user has written to `bigquery` questions

In [11]:
# your code here
bigquery_experts_query =  \
"""
SELECT pa.owner_user_id AS user_id, COUNT(1) AS number_of_answers
    FROM `bigquery-public-data.stackoverflow.posts_questions` AS pq
        INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS pa
            ON pq.id = pa.parent_id
    WHERE pq.tags LIKE '%bigquery%'
    GROUP BY pa.owner_user_id
    HAVING number_of_answers > 0
"""
bigquery_experts_results = stack_overflow.query_to_pandas_safe(bigquery_experts_query, max_gb_scanned=50)

print(bigquery_experts_results.head())
q_5.check()

     user_id  number_of_answers
0   266531.0                  4
1  4318535.0                  2
2  1599619.0                  1
3  3731823.0                  1
4  1970514.0                  1


<IPython.core.display.Javascript object>

None

In [12]:
# q_5.hint()
# q_5.solution()

# Building A More Generally Useful Service

How could you convert what you've done so it's a general function a website could call on the backend to get experts on any topic?  

Think about it and then check the solution below.

In [15]:
def expert_finder(topic, bigQueryHelper):
    '''
    Returns a DataFrame with the user IDs who have written Stack Overflow answers on a topic.
    
    Inputs:
        topic: A string with the topic of interest
        bigQueryHelper: A BigQueryHelper object that specifies the connection to the Stack Overflow dataset

    Outputs:
        results: A DataFrame with columns for user_id and number_of_answers sorted by number_of_answers.
    '''
    query = f"""
            SELECT pa.owner_user_id AS user_id, COUNT(1) AS number_of_answers
                FROM `bigquery-public-data.stackoverflow.posts_questions` AS pq
                    INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS pa
                        ON pq.id = pa.parent_id
                WHERE pq.tags LIKE '%{topic}%'
                GROUP BY pa.owner_user_id
                ORDER BY number_of_answers DESC
            """
    return bigQueryHelper.query_to_pandas_safe(query, max_gb_scanned=50)
    
print(expert_finder('tensorflow', stack_overflow).head())

     user_id  number_of_answers
0  3574081.0               1065
1   712995.0                478
2  1782792.0                407
3   992489.0                403
4   419116.0                400


In [16]:
# q_6.solution()

# Congratulations
You know all the key components to use BigQuery and SQL effectively. Your SQL skills are sufficient to unlock many of the world's large datasets.

Want to go play with your new powers?  Kaggle has BigQuery datasets available [here](https://www.kaggle.com/datasets?sortBy=hottest&group=public&page=1&pageSize=20&size=sizeAll&filetype=fileTypeBigQuery).

# Feedback
Bring any questions or feedback to the [Learn Discussion Forum](https://www.kaggle.com/learn-forum).


---
**[SQL Course Home Page](https://www.kaggle.com/learn/SQL)**

